In [ ]:
library(hh.analytics)
library(fs)
load.packages()

In [ ]:
##  Already in Snowflake
df_snowflake = suppressedReadCSV("C:\\Users\\jchang\\Downloads\\837-uploaded-snowflake.csv")

In [ ]:
df_files_all =
    list.files(
        "J:\\IU_Health_Complex\\837_FILES_IN\\",
        full.names = TRUE,
        recursive = TRUE
    ) %>% 
    data.frame(
        file_path = .
    ) %>% 
    mutate(
        file_name = file_path %>% str_extract("[^/]*$")
    ) %>% 
    filter(
        !file_name %in% df_snowflake$FILE_NAME
    )

In [ ]:
df_files_all %>%
    mutate(
        move = file_path %>% 
            map(
                .f = ~ fs::file_move(
                    path = .x,
                    new_path = .x %>% str_remove("/archive")
                )
            )
    )